In [1]:
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, GridSearchCV
import pickle 
import numpy as np

In [6]:
# SISTEMA DE RECOMENDACIÓN - ITEM-ITEM
def recomendacion_juego(item_id):
    """
    Sistema de recomendación item-item.

    Ingresando el ID de un producto, se devuelve una lista con 5 juegos recomendados similares al ingresado.

    Args:
    - item_id: El ID del producto para el cual se desean obtener recomendaciones.

    Returns:
    - dict: Un diccionario que contiene una lista de 5 juegos recomendados similares al ingresado.
      El diccionario tiene el formato {"Juegos similares al ID ingresado": {ID1: 'Nombre1', ID2: 'Nombre2', ...}}.

    Raises:
    - Si el DataFrame está vacío, devuelve un mensaje de error.
    - Si el item_id no existe en el DataFrame, devuelve un mensaje de error.
    - Si ocurre un error al cargar el archivo de similitud, intenta calcular la similitud del coseno
      entre los productos y proporciona recomendaciones basadas en eso.
    """
    # Leer el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')
    
    try:
        # Cargar la matriz de similitud de items_similarity
        with open('item_similarity.pkl', 'rb') as f:
            item_similarity_df = pickle.load(f)

        print("Se cargó la matriz precalculada.")

    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo: {e}")

        

        # Si el dataframe está vacío, devolver un mensaje de error
        if data.empty:
            return {
                "detail": [
                    {
                        "loc": ["string", 0],
                        "msg": "El DataFrame está vacío",
                        "type": "value_error"
                    }
                ]
            }

        # Si el item_id no existe en el DataFrame, devolver un mensaje de error
        if item_id not in data['item_id'].values:
            return {
                "detail": [
                    {
                        "loc": ["string", 0],
                        "msg": "El item_id proporcionado no existe",
                        "type": "value_error"
                    }
                ]
            }

        # Convertir la columna 'recommend' a valores numéricos
        data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

        # Crear la matriz de utilidad
        matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

        # Rellenar los valores faltantes con 0
        matrix.fillna(0, inplace=True)

        # Calcular la similitud del coseno
        item_similarity = cosine_similarity(matrix)

        # Convertir los resultados en un DataFrame
        item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)

        print("Se calculó la matriz de similitud.")

    if 'item_similarity_df' not in locals():
        # Obtener los juegos similares al juego dado
        similar_games = item_similarity_df[item_id].sort_values(ascending=False)

        # Obtener los 5 juegos más similares
        similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

        # Crear un diccionario con los ids y los nombres de los juegos
        recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

        return {"Juegos similares al id ingresado": recommended_games}
    else:
        # Obtener los juegos similares al juego dado
        similar_games = item_similarity_df[item_id].sort_values(ascending=False)

        # Obtener los 5 juegos más similares
        similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

        # Crear un diccionario con los ids y los nombres de los juegos
        recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

        return {"Juegos similares al id ingresado": recommended_games}

In [8]:
# llamada a la función
recomendacion_juego(30)

Se cargó la matriz precalculada.


{'Juegos similares al id ingresado': {250420: '8BitMMO',
  234490: 'Rush Bros.',
  390670: 'Steel Ocean',
  376570: 'WildStar',
  219680: 'Proteus'}}